# Homework 1 & 2

Elia Mirafiori VR537643

## Data Acquisition

### Acquisition of Calibration Images

To acquire the images, I've used my phone camera and as a pattern I've opted for a checkerboard 8x8 squares with 7x7 internal corners (the size of a square cell is 24mm per side). There have been collected 14 images to calibrate the camera with different poses and varying distances. All the images have been saved in JPG format.

### Choice and Acquisition of the Object

The object choosen for the 3D reconstruction is a Moai statue (it's a tissue holder). The reason why I chose it, it's because it rch in features and based on different light conditions, the detection and the matching of common points can vary widely. So, it's a good objection to test different setups.

## Libraries

In order to run the code, I needed a few 3-rd party libraries.

In [89]:
import os
import glob
import time

import cv2 as cv
import numpy as np

from matplotlib import pyplot as plt
from tabulate import tabulate

## Camera Calibration

As previously mentioned, I used a checkerboard to calibrate the camera. The criteria I employed were a maximum ceil of iterations (_30_) and a minimum required accuracy (_0.001_).

The procedure it's straight forward. I extrapolate world points (_3D_) and map it in image points (_2D_). To find the world points, I employed `cv.findChessboardCorners` that returns if the checkerboard corners have been found, if yes their coordinates get returned as well. Then, to get the image points I employed `cv.cornerSubPix` with the criteria specified before, a window size of _(11, 11)_ and a dead zone of _(-1, -1)_.

In the end, I employed the `cv.calibrateCamera` to retrieve the Camera matrix $\mathbf{K}$ and the Distortion Coefficients.

In [ ]:
def calibration(
    calibration_assets_path: str = "assets/calibration/phone/vertical",
    columns: int = 8,
    rows: int = 8,
    square_size: float = 1,
):
    """
    Calibration function
    """

    # Termination criteria for corner refinement (sub-pixel accuracy)
    # Stops when either:
    #  - max iterations are reached
    #  - or the desired accuracy is achieved
    criteria = (
        cv.TERM_CRITERIA_MAX_ITER + cv.TERM_CRITERIA_EPS,
        30,  # max number of iterations
        0.001,  # minimum required accuracy (epsilon)
    )

    # Chessboard configuration
    inner_corners = (columns - 1, rows - 1)  # number of INNER corners (columns, rows)

    # Prepare 3D object points in real-world coordinates
    # The chessboard lies on the Z = 0 plane
    objp = np.zeros((inner_corners[0] * inner_corners[1], 3), np.float32)

    # Generate grid and scale it by the square size (meter)
    # ':'	All rows, ':2'	First two columns only (index 0 and 1)
    objp[:, :2] = (
        np.mgrid[0 : inner_corners[0], 0 : inner_corners[1]].T.reshape(-1, 2)
        * square_size
    )

    # Containers for calibration points
    objpoints = []  # 3D points in real-world space (meter)
    imgpoints = []  # 2D points in image plane (pixels)

    # Load all calibration images from disk
    # Each image should show the same chessboard pattern
    images = glob.glob(f"{calibration_assets_path}/*.jpg")

    # Loop over each calibration image
    for img_path in images:
        print(f"Path:\n\t{img_path}")

        # Read image from disk (OpenCV loads images in BGR format)
        img_bgr = cv.imread(img_path)

        # Convert image to grayscale
        # Chessboard detection works on single-channel images
        img_gray = cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

        # Detect chessboard inner corners
        corners_found, corners = cv.findChessboardCorners(img_gray, inner_corners, None)

        print(f"Corners found:\n\t{corners_found}")

        # If the chessboard was successfully detected
        if corners_found:

            # Store the known 3D object points (real-world coordinates)
            # Same for every image, since the chessboard geometry is fixed
            objpoints.append(objp)

            # Refine corner positions to sub-pixel accuracy
            #
            # This improves calibration precision significantly
            #
            # (11, 11)  -> search window size
            # (-1, -1)  -> use default dead zone
            # criteria  -> termination criteria defined earlier
            corners_refined = cv.cornerSubPix(
                img_gray, corners, (11, 11), (-1, -1), criteria
            )

            # Store the refined 2D image points (pixel coordinates)
            imgpoints.append(corners_refined)

            # Visual feedback: draw detected corners on the image
            cv.drawChessboardCorners(
                img_bgr, inner_corners, corners_refined, corners_found
            )

            # Display the image briefly
            cv.imshow(
                "Calibration Image",
                cv.resize(
                    img_bgr,
                    (img_bgr.shape[1] // 4, img_bgr.shape[0] // 4),
                ),
            )
            cv.waitKey(500)  # display for 500 ms

    cv.destroyAllWindows()

    image_shape = cv.imread(images[0]).shape[:2][::-1]  # width, height

    rms_error, K, dist_coeffs, rot_vecs, trans_vecs = cv.calibrateCamera(
        objpoints, imgpoints, image_shape, None, None
    )

    # Compute the mean re-projection error (in pixels) over the calibration images
    errors = []

    for objp, imgp, rvec, tvec in zip(objpoints, imgpoints, rot_vecs, trans_vecs):
        projected, _ = cv.projectPoints(objp, rvec, tvec, K, dist_coeffs)
        projected = projected.reshape(-1, 2)
        imgp = imgp.reshape(-1, 2)

        # Euclidean pixel error per point
        err = np.linalg.norm(imgp - projected, axis=1)
        errors.append(err)

    errors = np.concatenate(errors)

    mean_error = np.mean(errors)
    std_error = np.std(errors)

    print(f"\nCamera Matrix K:\n{K}")
    print(f"Re-projection Error (in pixels):\n\t{rms_error}")
    # The error is good when it's under 0.08
    print(f"Mean reprojection error: {mean_error:.3f} px")
    print(f"Std dev reprojection error: {std_error:.3f} px")

    # Iterate over all images to show their individual poses
    for i, (r_vec, t_vec) in enumerate(zip(rot_vecs, trans_vecs)):
        R, _ = cv.Rodrigues(r_vec)
        print(f"\nImage {i} Pose")
        print(f"Rotation Matrix R:\n{R}")
        print(f"Translation t:\n{t_vec}")

    # Save calibration parameters
    param_path = os.path.join(calibration_assets_path, "calibration.npz")

    # Save several arrays into a single file in uncompressed .npz format
    np.savez(
        param_path,
        rms_error=rms_error,
        K=K,
        dist_coeffs=dist_coeffs,
        rot_vecs=rot_vecs,
        trans_vecs=trans_vecs,
    )

    return K, dist_coeffs

## Feature Detection & Matching

### Feature Detection

In order to reconstruct the object, I needed to pass 2 images of the object that differ by a translation and a rotation from each other. Given the input images, I've setup some detectors (ORB, BRISK, AKAZE, SIFT) to test their performances over the same images. Then, I needed the Camera matrix and the Distotion Coefficients to undistort the images and have a cleaner result.

I employed `cv.DETECTOR_create` to create a detector object and detector.detectAndCompute to start the detection and computation over the input images, this procedure finds **Keypoints** (points of interest like corners/edges) and calculates **Descriptors** (binary vectors that describe the area around the keypoint). These will be used to in all the further steps.

In [104]:
def feature_detection(
    img1_path: str,
    img2_path: str,
    detector=None,
    K=None,
    dist=None,
    debug: bool = False,
):
    """
    Detects features in two images using ORB and routes them to a matcher.
    """

    # Health check
    assert os.path.exists(img1_path), f"Left image not found: {img1_path}"
    assert os.path.exists(img2_path), f"Right image not found: {img2_path}"

    # Loads in GRAYSCALE because feature detection relies on intensity changes (gradients).
    # Color information is usually unnecessary for this and adds computational cost (3 channels vs 1).
    img1 = cv.imread(img1_path, cv.IMREAD_GRAYSCALE)  # Query image (left)
    img2 = cv.imread(img2_path, cv.IMREAD_GRAYSCALE)  # Train image (right)

    if K is not None and dist is not None:
        if debug:
            print("Undistorting images before detection...")

        # Refine camera matrix to avoid losing pixels at the edges
        # h, w = img1.shape[:2]
        # new_camera_matrix, roi = cv.getOptimalNewCameraMatrix(K, dist, (w,h), 1, (w,h))

        # Undistort
        img1 = cv.undistort(img1, K, dist, None, K)
        img2 = cv.undistort(img2, K, dist, None, K)

    # Initialize Detector
    # 'nfeatures=5000' is the maximum number of keypoints to retain.
    # The default is often 500, but 5000 is better for high-res images or detailed scenes
    # to ensure enough matches are found later.
    if detector is None:
        detector = cv.ORB_create(nfeatures=10000)

    # Detection & Description
    # detectAndCompute performs two steps:
    #   1. Detect: Finds 'Keypoints' (points of interest like corners/edges)
    #   2. Compute: Calculates 'Descriptors' (binary vectors that describe the area around the keypoint)
    # The 'mask=None' argument means we look for features in the entire image.
    kp1, des1 = detector.detectAndCompute(img1, None)
    kp2, des2 = detector.detectAndCompute(img2, None)

    # akaze = cv.AKAZE_create(threshold=0.0005)
    #
    # kp1, des1 = akaze.detectAndCompute(img1, None)
    # kp2, des2 = akaze.detectAndCompute(img2, None)

    if debug:
        print(f"Detected {len(kp1)} keypoints in left image.")
        print(f"Detected {len(kp2)} keypoints in right image.")

    # Safety Check
    # If an image has no texture (e.g., a blank wall), descriptors might be None.
    # Proceeding without this check would cause the matchers to crash.
    if des1 is None or des2 is None:
        raise RuntimeError(
            "Descriptors are None. Try different images or a different feature extractor."
        )

    return img1, kp1, des1, img2, kp2, des2

### Feature Matcher

After the feature detection, I needed to match the keypoints of the input images. To do so, I've used a set of matchers (Brute-Force, FLANN). In the end, I compared the results of the pair _DETECTOR-MATCHER_. I employed `cv.NAMEMatcher` to create the matcher object.

For the Brute-Force matcher, I've sorted the matches and filtered out the ones with an higher distance.

Meanwhile, for the FLANN matcher, I've applied the KNN match and the Lowe's Ratio test.

In order to keep a reasonable amount of points, I filtered out and kept only at max 5000 matches.

In [ ]:
def feature_matcher(
    img1,
    kp1,
    des1,
    img2,
    kp2,
    des2,
    matcher=None,
    max_matches: int = 5000,
    ratio_test: float = 0.70,
):
    """
    Matches descriptors using the provided matcher.
    Automatically applies ratio test for FLANN matchers and optionally for BFMatcher.
    """

    # Initialize Matcher
    # cv.NORM_HAMMING: Essential for binary descriptors (ORB, BRIEF).
    # It calculates distance by counting the number of differing bits (XOR operation).
    # crossCheck=True: This enforces a mutual match.
    # Feature A in img1 must match B in img2, AND Feature B in img2 must match A in img1.
    # This filters out many false positives automatically.
    if matcher is None:
        matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Check if matcher is FLANN or BF
    matcher_type = type(matcher).__name__

    good_matches = []

    # Perform Matching
    # A DMatch object has four main attributes:
    # Match(
    #    queryIdx,  # index of the descriptor in the first set (des1)
    #    trainIdx,  # index of the descriptor in the second set (des2)
    #    imgIdx,    # index of the training image (used with multiple images)
    #    distance   # distance between the two descriptors
    # )
    if matcher_type == "FlannBasedMatcher":
        # FLANN: use knnMatch + ratio test
        matches = matcher.knnMatch(des1, des2, k=2)

        # Need to draw only good matches, so create a mask
        matchesMask = [[0, 0] for i in range(len(matches))]

        # Ratio test as per Lowe's paper
        for i, (m, n) in enumerate(matches):
            if m.distance < ratio_test * n.distance:
                good_matches.append(m)
    else:
        # BFMatcher
        if getattr(matcher, "crossCheck", True):
            # Cross-checked BF: use match()
            matches = matcher.match(des1, des2)
            # Lower distance = more similar descriptors = better match.
            good_matches = sorted(matches, key=lambda x: x.distance)
        else:
            # Non-cross-checked BF: use knnMatch + ratio test
            matches = matcher.knnMatch(des1, des2, k=2)

            # Ratio test as per Lowe's paper
            for m, n in matches:
                if m.distance < ratio_test * n.distance:
                    good_matches.append(m)
            # Lower distance = more similar descriptors = better match.
            good_matches = sorted(good_matches, key=lambda x: x.distance)

    # Selection
    # We slice the list to keep only the top N matches.
    # This removes weak matches that might just be noise or repetitive textures.
    good_matches = good_matches[: min(max_matches, len(good_matches))]

    print(
        f"Using {len(good_matches)} best matches for fundamental matrix estimation."
    )

    return good_matches

## Visualize Matcher

To visualize the Feature Detection and Matching, I've setup a simple function that requries the input images with their keypoints and matches. Then, I employed `cv.drawMatches` to draw the matches.

In [106]:
def visualize_matcher(img1, kp1, img2, kp2, matches):

    # Visualization
    matches_to_show = min(100, len(matches))

    # drawMatches creates a new image containing both img1 and img2 side-by-side
    # and draws lines connecting the matched keypoints.
    img_matches = cv.drawMatches(
        img1,
        kp1,
        img2,
        kp2,
        matches[:matches_to_show],
        None,
        flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,  # Only draw matched points, not all detected ones
    )

    # Plotting
    plt.figure(figsize=(15, 10))
    plt.title("Feature matches (subset)")
    plt.imshow(img_matches)
    plt.axis("off")
    plt.show()

## Keypoint coordinates from Matches

In the next steps, I needed the coordinates of the matches, so with this function I retrieved them.

In [107]:
def get_keypoint_coords_from_matches(kp1, kp2, matches):
    """
    Helper to extract coordinates from matches.
    """

    # Point Extraction
    # Convert matched keypoints into simple numpy arrays of (x, y) coordinates.
    # pts1: Points in the Left Image (from queryIdx)
    # pts2: Points in the Right Image (from trainIdx)
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    return pts1, pts2

## Epipolar Geometry

### Fundamental Matrix

In order to derive the Essential matrix $\mathbf{E}$, I had two possible approaches. The **Indirect Approach** requires the Fundamental matrix $\mathbf{F}$, then with the Camera matrix found previously I could compute it by doing $\mathbf{E} = \mathbf{K}^\top \mathbf{F} \mathbf{K}$. The results obtained with this approach were pretty similar to the ones got with the **Direct Approach**. The latter is the one I've implemented below.

The Fundamental matrix has been computed with **RANSAC** (threshold of _1_ and a confidence of _0.999_). I employed the `cv.findFundamentalMat` function that gave me also a mask to filter out the outliers.

The remaining polished points were used to compute the Essential matrix.

In [108]:
def estimate_fundamental_matrix(
    pts1,
    pts2,
    ransac_thresh=1.0,
    confidence=0.999,
):
    """
    Estimates the Fundamental Matrix F using RANSAC and visualizes inliers.
    """
    # Fundamental Matrix Estimation
    # This calculates the relationship between pixel coordinates in two views.
    # cv.FM_RANSAC: Uses Random Sample Consensus to ignore outliers (bad matches).
    # ransacReprojThreshold: The maximum distance (in pixels) a point can be from the epipolar line to be considered an inlier.
    F, mask = cv.findFundamentalMat(
        pts1,
        pts2,
        cv.FM_RANSAC,
        ransacReprojThreshold=ransac_thresh,
        confidence=confidence,
    )

    if F is None:
        raise RuntimeError("Fundamental matrix estimation failed")

    # Filtering Inliers
    # "mask" is a list of 0s (outliers) and 1s (inliers).
    # We flatten it to a 1D array of booleans.
    mask = mask.ravel().astype(bool)

    # Select ONLY the points that agree with the Fundamental Matrix geometry.
    pts1_in = pts1[mask]
    pts2_in = pts2[mask]


    return F, pts1_in, pts2_in, mask

### Essential Matrix

As mentioned before, I've computed the Essential matrix by employing the `cv.findEssentialMat` function. It required as an input the matched points and the Camera matrix. I've computed it with **RANSAC** (threshold of _1_ and a confidence level of _0.999_). Also this function returned a mask to filter out the outliers.

The remaining polished points were used in the next steps.

In [109]:
def estimate_essential_matrix(
    pts1,
    pts2,
    K,
    prob=0.999,
    threshold=1.0,
):
    """
    Estimates the Essential Matrix E using RANSAC and the Camera Matrix K.
    """

    # Essential Matrix Estimation
    # method=cv.RANSAC: Standard RANSAC
    # prob=0.999: Confidence level (higher is better for E)
    # threshold=1.0: Max distance from epipolar line (in pixels)
    E, mask = cv.findEssentialMat(
        pts1,
        pts2,
        K,
        method=cv.RANSAC,
        prob=prob,
        threshold=threshold,
    )

    if E is None:
        raise RuntimeError("Essential matrix estimation failed")

    # Flatten the mask
    # "mask" is a list of 0s (outliers) and 1s (inliers).
    # We flatten it to a 1D array of booleans.
    mask = mask.ravel().astype(bool)

    # Select ONLY the points that agree with the Fundamental Matrix geometry.
    pts1_in = pts1[mask]
    pts2_in = pts2[mask]

    return E, pts1_in, pts2_in, mask

### Epipolar Geometry Visualization

For every point in one image, the Fundamental Mmtrix determines a line in the other image on which the corresponding point **must** lie.

To accomplish this task, I employed the `cv.computeCorrespondEpilines` function on both input images, then I drew the lines over them to visualize it.

In [110]:
def draw_epipolar_lines(img1, pts1, img2, pts2, F):
    """
    Visualizes epipolar lines.
    For every point in one image, the Fundamental Matrix determines a line in the other image
    on which the corresponding point MUST lie.
    """

    # Right-to-Left Epilines
    # We take points from the RIGHT image (pts2) and calculate where their
    # corresponding epipolar lines should be in the LEFT image.
    # The '2' argument tells OpenCV the points are from the second image.
    # F maps p2 -> line1 (l = F.T * p2)
    lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(
        -1, 3
    )  # Reshape to N x 3 (a, b, c for line ax + by + c = 0)

    # Draw these lines on img1 (Left Image)
    # img5 will show lines on Left Image, img6 shows the points on Right Image
    img5, img6 = drawlines(img1, img2, lines1, pts1, pts2)

    # Left-to-Right Epilines
    # We take points from the LEFT image (pts1) and find lines in the RIGHT image.
    # The '1' argument tells OpenCV the points are from the first image.
    # F maps p1 -> line2 (l = F * p1)
    lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)

    # Draw these lines on img2 (Right Image)
    # img3 will show lines on Right Image, img4 shows the points on Left Image
    img3, img4 = drawlines(img2, img1, lines2, pts2, pts1)

    # Visualization
    plt.figure(figsize=(15, 10))
    plt.subplot(121), plt.imshow(img5)
    plt.title("Epilines on Left Image"), plt.axis("off")
    plt.subplot(122), plt.imshow(img3)
    plt.title("Epilines on Right Image"), plt.axis("off")
    plt.show()


def drawlines(img1, img2, lines, pts1, pts2):
    """
    Helper function to draw lines on one image and matching points on the other.

    Args:
        img1: Image on which we draw the epilines (lines).
        img2: Image on which we draw the points (pts2) that generated those lines.
        lines: The epipolar lines coefficients (a, b, c) where ax + by + c = 0
        pts1: Points in img1 (that should fall ON the lines).
        pts2: Points in img2 (that generated the lines via F).
    """

    # Get image dimensions (rows, cols) to calculate start/end points of lines
    r, c = img1.shape

    # Convert to BGR so we can draw colored lines/circles
    img1 = cv.cvtColor(img1, cv.COLOR_GRAY2BGR)
    img2 = cv.cvtColor(img2, cv.COLOR_GRAY2BGR)

    # Iterate through lines and points simultaneously
    # 'lines' corresponds to pts2 (source) and matches pts1 (destination)
    for r, pt1, pt2 in zip(lines, pts1, pts2):

        # Random color for each pair to distinguish them
        color = tuple(np.random.randint(0, 255, 3).tolist())

        # --- Line Calculation ---
        # Epipolar line equation: ax + by + c = 0
        # We need two points (x0, y0) and (x1, y1) to draw the line segment.
        # r[0]=a, r[1]=b, r[2]=c

        # Point 1 (Left edge, x=0):
        # a(0) + by + c = 0  =>  by = -c  =>  y = -c/b
        x0, y0 = map(int, [0, -r[2] / r[1]])

        # Point 2 (Right edge, x=c (width)):
        # a(c) + by + c = 0  =>  by = -(c + ac)  =>  y = -(c + ac)/b
        # Note: variable 'c' here is image width (cols), r[2] is line coefficient 'c'
        x1, y1 = map(int, [c, -(r[2] + r[0] * c) / r[1]])

        # Draw the epipolar line on the target image
        img1 = cv.line(img1, (x0, y0), (x1, y1), color, 1)

        # Draw the corresponding point on the target image (should be ON the line)
        # Note: pt1 needs to be integer tuple for cv.circle
        img1 = cv.circle(img1, tuple(np.int32(pt1)), 5, color, -1)

        # Draw the source point on the source image (just for reference)
        img2 = cv.circle(img2, tuple(np.int32(pt2)), 5, color, -1)

    return img1, img2

## Recover Pose

In order to compute the **Projection** matrices $\mathbf{P1}$ and $\mathbf{P2}$, I first needed to recover the **Rotation** matrix $\mathbf{R}$ and the Translation vector $\mathbf{t}$. I employed `cv.recoverPose` from OpenCV, it uses the **inliers** points (_pts1_, _pts2_) to check which of the 4 solutionsis valid.

```python
num_points, R, t, mask = cv.recoverPose(E, pts1_in, pts2_in, K)
```

`cv.recoverPose` returns a mask that I used to filter points again. This approach helped a lot to get a more polished reconstruction.

```python
mask_pose = mask.ravel().astype(bool)
pts1_valid = pts1_in[mask_pose]
pts2_valid = pts2_in[mask_pose]
```

## Projection Matrices

As previously stated, to compute the Projection matrices I've retrieved the Rotation matrix and the Translation vector. Then, I've computed them by using the following formula $$\mathbf{P1} = \mathbf{K} \left[ \mathbf{I} \mid \mathbf{0} \right], \quad
\mathbf{P2} = \mathbf{K} \left[ \mathbf{R} \mid \mathbf{t} \right]$$

In [111]:
def estimate_projection_matrices(K, R, t):
    """
    Computes P1 and P2 from the Essential Matrix using recoverPose.
    """
    # Construct P1 (Origin)
    # P1 = K @ [I | 0]
    # Identity matrix (3x3) concatenated with a zero column (3x1)
    I = np.eye(3)
    zeros = np.zeros((3, 1))
    P1 = np.dot(K, np.hstack((I, zeros)))

    # Construct P2 (New View)
    # P2 = K @ [R | t]
    P2 = np.dot(K, np.hstack((R, t)))

    # Return P1, P2 AND the points that actually define this pose
    return P1, P2

## Triangulation

To build the 3D reconstruction, I needed to pass from the 2D image points to the 3D real-world points.

Given the Projection matrices and the inliers, I employed `cv.triangulatePoints` to get a _4xN_ matrix of homogeneous coordinates _(x, y, z, w)_. Then, I had to convert **Homogeneous** (_4D_) coordinates to **Euclidean** (_3D_) coordinates by dividing _x_, _y_, and _z_ by _w_.

In [112]:
def triangulate_3d_points(P1, P2, pts1, pts2):
    """
    Triangulates 2D points from two views into 3D space.
    """

    # Transpose points
    # cv.triangulatePoints expects data in shape (2, N),
    # but our points are currently (N, 2).
    pts1_t = pts1.T
    pts2_t = pts2.T

    # Triangulate
    # This returns a 4xN matrix of homogeneous coordinates (x, y, z, w)
    points_4d = cv.triangulatePoints(P1, P2, pts1_t, pts2_t)

    # Convert Homogeneous (4D) to Euclidean (3D)
    # We must divide x, y, and z by w.
    points_3d = points_4d[:3, :] / points_4d[3, :]

    # Transpose back to (N, 4) and take the first 3 columns (x, y, z)
    points_3d = points_3d[:3].T

    return points_3d

## Reprojection error

This step is just a quantitative result and I used it to compute the average re-projection error (RMSE) in pixels.

In [113]:
def compute_reprojection_error(P1, P2, points_3d, pts1, pts2):
    """
    Computes the average re-projection error (RMSE) in pixels.
    """
    # Convert 3D points to homogeneous coordinates (4xN)
    # Shape becomes (4, N) -> [[x, y, z, 1], ...]
    points_3d_hom = np.hstack((points_3d, np.ones((points_3d.shape[0], 1)))).T

    # Project into Image 1
    # P1 is 3x4, points_3d_hom is 4xN -> projected_1 is 3xN
    projected_1_hom = np.dot(P1, points_3d_hom)

    # Normalize by the last row (Z/W) to get pixel coordinates (u, v)
    projected_1 = projected_1_hom[:2] / projected_1_hom[2]

    # Calculate Euclidean distance between observed and projected points
    # pts1 is (N, 2), projected_1.T is (N, 2)
    error_1 = np.linalg.norm(pts1 - projected_1.T, axis=1)

    # Project into Image 2
    projected_2_hom = np.dot(P2, points_3d_hom)
    projected_2 = projected_2_hom[:2] / projected_2_hom[2]
    error_2 = np.linalg.norm(pts2 - projected_2.T, axis=1)

    # Compute Mean Error
    mean_error_1 = np.mean(error_1)
    mean_error_2 = np.mean(error_2)
    total_mean_error = (mean_error_1 + mean_error_2) / 2.0

    return mean_error_1, mean_error_2, total_mean_error

## 3D Reconstruction

I used this step to visualize the 3D point cloud using Matplotlib with depth-based coloring.

In [114]:
def plot_3d_point_cloud(points_3d):
    """
    Visualizes the 3D point cloud using Matplotlib with depth-based coloring.
    """
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection="3d")

    # Extract X, Y, Z columns
    X = points_3d[:, 0]
    Y = points_3d[:, 1]
    Z = points_3d[:, 2]

    # c=Z          -> Maps the color to the Z value (depth)
    # cmap='viridis' -> Sets the gradient scheme (try 'plasma', 'inferno', 'jet')
    # depthshade=True -> Optional: adds shading to enhance 3D perception
    scatter = ax.scatter(X, Y, Z, c=Z, cmap="viridis", marker=".", s=2, depthshade=True)

    # Add a color bar to interpret the depth
    cbar = plt.colorbar(scatter, ax=ax, pad=0.1, shrink=0.6)
    cbar.set_label("Depth (Z)")

    ax.set_xlabel("X Axis")
    ax.set_ylabel("Y Axis")
    ax.set_zlabel("Z Axis")
    ax.set_title("3D Reconstruction (Colored by Depth)")

    # (Keep your existing scaling logic)
    max_range = (
        np.array([X.max() - X.min(), Y.max() - Y.min(), Z.max() - Z.min()]).max() / 2.0
    )
    mid_x = (X.max() + X.min()) * 0.5
    mid_y = (Y.max() + Y.min()) * 0.5
    mid_z = (Z.max() + Z.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)

    # Look from the bottom
    ax.view_init(elev=-90, azim=-90)

    plt.show()

## Main Code

Here I've setup the core of my software, you can see the login and how I managed to test some pairs of Detectors and Matchers.

In [115]:
if __name__ == "__main__":
    # Check if we need to run calibration, or just load it
    camera_path = "phone/vertical"
    calib_path = f"assets/calibration/{camera_path}/calibration.npz"
    debug = True
    detectors_matchers = [
        # ORB with lots of features, BFMatcher using Hamming (binary)
        (
            cv.ORB_create(nfeatures=10000),
            cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True),
        ),
        # ORB with FLANN LSH
        (
            cv.ORB_create(nfeatures=10000),
            cv.FlannBasedMatcher(
                dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1), {}
            ),  # LSH for binary descriptors
        ),
        # AKAZE with custom threshold, BFMatcher using Hamming
        (
            cv.AKAZE_create(threshold=0.0005),
            cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True),
        ),
        # BRISK with BFMatcher using Hamming (binary)
        (cv.BRISK_create(), cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)),
        # SIFT with BFMatcher using L2 (float descriptors)
        (cv.SIFT_create(), cv.BFMatcher(cv.NORM_L2, crossCheck=True)),
        # SIFT with FLANN KD-Tree
        (
            cv.SIFT_create(),
            cv.FlannBasedMatcher(
                dict(algorithm=1, trees=5), dict(checks=50)
            ),  # KD-Tree for float descriptors
        ),
    ]

    results = []

    K, dist = None, None

    print("Running calibration...")
    K, dist = calibration(
        calibration_assets_path=f"assets/calibration/{camera_path}",
        square_size=0.024,
        debug=debug,
    )

    if debug:
        print(f"Camera Matrix K:\n{K}\n")
        print(f"Distortion Coefficients:\n{dist}\n")

    for detector, matcher in detectors_matchers:
        start_time = time.perf_counter()

        if debug:
            detector_type = type(detector).__name__
            matcher_type = type(matcher).__name__
            print(f"\n\nDetector: {detector_type}")
            print(f"Matcher: {matcher_type}")

        ### Feature Extraction and Matching ###

        if debug:
            print("\n\n### Feature Extraction and Matching ###\n")

        # Run feature detection WITH undistortion
        img1, kp1, des1, img2, kp2, des2 = feature_detection(
            img1_path=f"assets/{camera_path}/moai_view1.jpg",
            img2_path=f"assets/{camera_path}/moai_view2.jpg",
            K=K,
            dist=dist,
            detector=detector,
            debug=debug,
        )

        n_keypoints = len(kp1)

        matches = feature_matcher(
            img1, kp1, des1, img2, kp2, des2, matcher=matcher, debug=debug
        )

        n_matches = len(matches)

        end_time = time.perf_counter()
        elapsed_time = end_time - start_time

        if debug:
            print(f"Time elapsed: {elapsed_time:.4f} s\n")
            print(
                f"Number of keypoints: {len(kp1)}, Number of matches: {len(matches)}\n"
            )

        if debug:
            visualize_matcher(img1, kp1, img2, kp2, matches)

        ### Epipolar Geometry Estimation ###

        if debug:
            print("\n\n### Epipolar Geometry Estimation ###\n")

        # Extract matched points
        pts1, pts2 = get_keypoint_coords_from_matches(kp1, kp2, matches)

        if debug:
            print(f"Initial Matches: {len(pts1)}")

        F, pts1_in, pts2_in, mask = estimate_fundamental_matrix(
            pts1, pts2, ransac_thresh=1.0, confidence=0.999
        )

        if debug:
            print(f"F inliers: {len(pts1_in)} / {len(pts1)}")
            print(f"Fundamental Matrix:\n{F}\n")

        # Essential matrix directly
        # We're using the inliers found with F, new inliers will be more polished
        E, pts1_in, pts2_in, mask = estimate_essential_matrix(pts1_in, pts2_in, K)

        n_inliers = len(pts1_in)

        if debug:
            print(f"E inliers: {len(pts1_in)} / {len(pts1)}")
            print(f"Essential Matrix:\n{E}\n")

        # Recover pose
        # This extracts the rotation and translation.
        # It uses the points (pts1, pts2) to check which of the 4 solutions is valid.
        # pts1 and pts2 should be the INLIERS from your previous steps.
        num_points, R, t, mask = cv.recoverPose(E, pts1_in, pts2_in, K)

        # Filter points again using the pose mask
        mask_pose = mask.ravel().astype(bool)
        pts1_valid = pts1_in[mask_pose]
        pts2_valid = pts2_in[mask_pose]

        if debug:
            print(f"Recovered Pose with {num_points} valid points.")
            print(f"Rotation matrix R:\n{R}\n")
            print(f"Translation vector t:\n{t}\n")
            print(f"RecoverPose kept {num_points} points (Cheirality check)")

        if debug:
            draw_epipolar_lines(img1, pts1_valid, img2, pts2_valid, F)

        ### Triangulation and 3D Reconstruction ###

        if debug:
            print("\n\n### Triangulation and 3D Reconstruction ###\n")

        P1, P2 = estimate_projection_matrices(K, R, t)

        if debug:
            print(f"Projection Matrix 1:\n{P1}\n")
            print(f"Projection Matrix 2:\n{P2}\n")

        points_3d = triangulate_3d_points(P1, P2, pts1_valid, pts2_valid)

        if debug:
            print(
                "Valid points Z range:",
                np.min(points_3d[:, 2]),
                np.max(points_3d[:, 2]),
            )

        # Positive depth filter
        mask_z = points_3d[:, 2] > 0

        points_3d = points_3d[mask_z]
        pts1_used = pts1_valid[mask_z]
        pts2_used = pts2_valid[mask_z]

        # Filtering out extreme depth outliers using percentile
        z = points_3d[:, 2]
        upper = np.percentile(z, 97)

        mask_depth = z < upper

        points_3d = points_3d[mask_depth]
        pts1_used = pts1_used[mask_depth]
        pts2_used = pts2_used[mask_depth]

        n_3d_points = points_3d.shape[0]

        if debug:
            print(f"Generated {len(points_3d)} 3D points.")

        # Calculate quantitative error
        mean_error_1, mean_error_2, total_mean_error = compute_reprojection_error(
            P1, P2, points_3d, pts1_used, pts2_used
        )

        if debug:
            print(f"Re-projection Error Camera 1: {mean_error_1:.4f} px")
            print(f"Re-projection Error Camera 2: {mean_error_2:.4f} px")
            print(f"Total Mean Re-projection Error: {total_mean_error:.4f} px")

        if debug:
            plot_3d_point_cloud(points_3d)

        # Append results
        results.append(
            [
                detector.__class__.__name__,
                matcher.__class__.__name__,
                n_keypoints,
                n_matches,
                n_inliers,
                n_3d_points,
                f"{total_mean_error:.2f} px",
                f"{elapsed_time:.2f} s",
            ]
        )

    print(
        tabulate(
            results,
            headers=[
                "Detector",
                "Matcher",
                "#Keypoints",
                "#Matches",
                "#Inliers",
                "#3D Points",
                "Reproj Error",
                "Time",
            ],
            tablefmt="github",
        )
    )


Running calibration...


TypeError: calibration() got an unexpected keyword argument 'debug'